In [ ]:
# Copyright 2021 Seb Seager

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#    http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


In [ ]:
import numpy as np
import qiskit
from qiskit.quantum_info import Statevector

In [57]:
def keywords_satisfied(text, keywords):
  ans = [0] * len(keywords)
  for j in range(len(text)):
    for i in range(len(keywords)):
      if keywords[i] == text[j]:
        ans[i] = 1
  return sum(ans)
  

In [58]:
def oracle(article, keywords):
  n = keywords_satisfied(article, keywords) 
  return 1/(2**(len(keywords) - n))

In [59]:
# change article.text to whatever the API defines
def encode_articles(articles, keywords):
  sum = 0
  results = []
  for i in range(len(articles)):
    results.append(oracle(articles[i], keywords))
    sum += results[i]**2
  for i in range(len(articles)):
  	results[i] = (1/(sum**(1/2)))* results[i]
  return results

In [60]:
import requests

# api call
x = requests.get('https://newsapi.org/v2/top-headlines?country=us&apiKey=b72aad9271604a1b888e9479c9de4fb7')
articles = dict(x.json())['articles']

In [61]:
# process articles from api
test_articles = []
for a in articles:
  if a['content']:
    new_content = a['content'].lower().replace(".,![]{}-—_;':?", "").split()
    new_title = a['title']
    new_url = a['url']
    test_articles.append((new_title, new_content, new_url))

In [76]:
# keywords to check for - THIS IS THE USER INPUT
test_keywords = ["test", "hello", "monday"]

# encoding step
def find_article(articles, keywords):
  encoded_vector = list(encode_articles([x[1] for x in articles], keywords))
  encoded_vector = np.array(encoded_vector)
  state_vector = Statevector(data=encoded_vector)
  chosen_article = articles[int(state_vector.measure()[0], 2)]
  return chosen_article[0], chosen_article[2]

In [85]:
# example of getting an article
received_article = find_article(test_articles, test_keywords)
print(received_article)

("'Overtime, we will see the NFT market broaden,' says Ripple’s CTO David Schwartz - Cointelegraph", 'https://cointelegraph.com/news/overtime-we-will-see-the-nft-market-broaden-says-ripple-s-cto-david-schwartz')
